In [1]:
import timm
import torch
import torchinfo


c:\Users\Ingo\Desktop\Code Stuff\mae\mae\.mae-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
vit = timm.create_model('vit_base_patch16_224', pretrained=False)

# add classification head
vit.head = torch.nn.Linear(in_features=768, out_features=1000, bias=True)

torch.save(vit.state_dict(), '../mae_timm_vit_base.pth')


In [5]:
checkpoint = torch.load('../mae_finetuned_vit_base.pth', map_location='cpu')
new_state_dict = {}
for key, value in checkpoint['model'].items():
    # Rename keys to match the model's expected keys
    new_key = key.replace('fc_norm', 'norm')  # Adjust based on the specific mismatch
    new_state_dict[new_key] = value

vit.load_state_dict(new_state_dict, strict=True)

<All keys matched successfully>

In [6]:
torchinfo.summary(vit, input_size=(1, 3, 224, 224))

c:\Users\Ingo\Desktop\Code Stuff\mae\mae\.mae-env\Lib\site-packages\timm\models\vision_transformer.py:92: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  x = F.scaled_dot_product_attention(


Layer (type:depth-idx)                   Output Shape              Param #
VisionTransformer                        [1, 1000]                 152,064
├─PatchEmbed: 1-1                        [1, 196, 768]             --
│    └─Conv2d: 2-1                       [1, 768, 14, 14]          590,592
│    └─Identity: 2-2                     [1, 196, 768]             --
├─Dropout: 1-2                           [1, 197, 768]             --
├─Identity: 1-3                          [1, 197, 768]             --
├─Identity: 1-4                          [1, 197, 768]             --
├─Sequential: 1-5                        [1, 197, 768]             --
│    └─Block: 2-3                        [1, 197, 768]             --
│    │    └─LayerNorm: 3-1               [1, 197, 768]             1,536
│    │    └─Attention: 3-2               [1, 197, 768]             2,362,368
│    │    └─Identity: 3-3                [1, 197, 768]             --
│    │    └─Identity: 3-4                [1, 197, 768]           